### 6-1. 環境設定

In [7]:
from dotenv import load_dotenv
import os

# .env ファイルを読み込む
path_env="C:\\Users\\ktgu1\\.env"
load_dotenv(path_env)

# 環境変数を取得
langsmith_tracing_v2 = os.getenv("LANGCHAIN_TRACING_V2")
langsmith_endpoint = os.getenv("LANGSMITH_ENDPOINT")
langsmith_api_key = os.getenv("LANGSMITH_API_KEY")
langsmith_project = os.getenv("LANGSMITH_PROJECT")

# Langsmithがただしく読み込めているか確認。なぜかtracingがNoneだができてる
print(f"LANGSMITH_TRACING_V2: {langsmith_tracing_v2}")
print(f"LANGSMITH_ENDPOINT: {langsmith_endpoint}")
print(f"LANGSMITH_PROJECT: {langsmith_project}")

LANGSMITH_TRACING_V2: None
LANGSMITH_ENDPOINT: https://api.smith.langchain.com
LANGSMITH_PROJECT: agent-book


In [10]:
%pip install langchain-core==0.3.0 langchain-openai==0.2.0 \
    langchain-community==0.3.0 GitPython==3.1.43 \
    langchain-chroma==0.1.4 tavily-python==0.5.0 pydantic==2.10.6

  Using cached langchain_core-0.3.0-py3-none-any.whl.metadata (6.2 kB)
  Using cached langchain_openai-0.2.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached langchain_community-0.3.0-py3-none-any.whl.metadata (2.8 kB)
  Using cached langchain_chroma-0.1.4-py3-none-any.whl.metadata (1.6 kB)
  Using cached tavily_python-0.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached pydantic-2.10.6-py3-none-any.whl.metadata (30 kB)
  Using cached langsmith-0.1.147-py3-none-any.whl.metadata (14 kB)
  Using cached openai-1.86.0-py3-none-any.whl.metadata (25 kB)
  Using cached tiktoken-0.9.0-cp312-cp312-win_amd64.whl.metadata (6.8 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached langchain-0.3.25-py3-none-any.whl.metadata (7.8 kB)
  Using cached pydantic_settings-2.9.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached chromadb-0.5.23-py3-none-any.whl.metadata (6.8 kB)
  Using cached fastapi-0.115.12-py3-none-any.whl.metadata (27 kB)
  Using cached pydantic_

### 6-2. RAG実行

In [11]:
from langchain_community.document_loaders import GitLoader


def file_filter(file_path: str) -> bool:
    return file_path.endswith(".mdx")


loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter,
)

documents = loader.load()
print(len(documents))

417


In [12]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
db = Chroma.from_documents(documents, embeddings)

In [14]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。

文脈: """
{context}
"""

質問: {question}
''')

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

retriever = db.as_retriever()

chain = {
    "question": RunnablePassthrough(),
    "context": retriever,
} | prompt | model | StrOutputParser()

chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクルの各段階を簡素化します。具体的には、以下のような機能を提供しています。\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティの統合を使用してアプリケーションを構築できます。LangGraphを利用することで、状態を持つエージェントを構築し、ストリーミングや人間の介入をサポートします。\n\n2. **生産化**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換することができます。\n\nLangChainは、LLMや関連技術（埋め込みモデルやベクトルストア）に対する標準インターフェースを実装し、数百のプロバイダーと統合しています。また、複数のオープンソースライブラリで構成されており、ユーザーは必要に応じてコンポーネントを選択して使用できます。'

### 6-3. 検索クエリの工夫

In [15]:
hypothetical_prompt = ChatPromptTemplate.from_template("""\
次の質問に回答する一文を書いてください。

質問: {question}
""")

hypothetical_chain = hypothetical_prompt | model | StrOutputParser()

In [16]:
hyde_rag_chain = {
    "question": RunnablePassthrough(),
    "context": hypothetical_chain | retriever,
} | prompt | model | StrOutputParser()

hyde_rag_chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。LangChainは、開発、運用、デプロイの各段階を簡素化することを目的としています。具体的には、以下のような特徴があります。\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティの統合を使用してアプリケーションを構築できます。また、LangGraphを利用して、状態を持つエージェントを構築することができます。\n\n2. **運用**: LangSmithを使用してアプリケーションを監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイ**: LangGraphアプリケーションを本番環境向けのAPIやアシスタントに変換することができます。\n\nLangChainは、さまざまなプロバイダーと統合し、標準インターフェースを提供することで、開発者が異なるコンポーネントを簡単に切り替えたり、組み合わせたりできるようにしています。また、複雑なアプリケーションのオーケストレーションをサポートするためのLangGraphや、アプリケーションの可視化と評価を行うLangSmithといったツールも提供しています。'

### 複数の検索クエリの生成

In [17]:
from pydantic import BaseModel, Field


class QueryGenerationOutput(BaseModel):
    queries: list[str] = Field(..., description="検索クエリのリスト")


query_generation_prompt = ChatPromptTemplate.from_template("""\
質問に対してベクターデータベースから関連文書を検索するために、
3つの異なる検索クエリを生成してください。
距離ベースの類似性検索の限界を克服するために、
ユーザーの質問に対して複数の視点を提供することが目標です。

質問: {question}
""")

query_generation_chain = (
    query_generation_prompt
    | model.with_structured_output(QueryGenerationOutput)
    | (lambda x: x.queries)
)

In [18]:
multi_query_rag_chain = {
    "question": RunnablePassthrough(),
    "context": query_generation_chain | retriever.map(),
} | prompt | model | StrOutputParser()

multi_query_rag_chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。LangChainは、開発、運用、デプロイの各段階を簡素化することを目的としています。具体的には、以下のような機能を提供しています。\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティの統合を使用してアプリケーションを構築できます。また、LangGraphを利用して、状態を持つエージェントを構築し、ストリーミングや人間の介入をサポートします。\n\n2. **運用**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換できます。\n\nLangChainは、さまざまなモデルや関連コンポーネントの標準インターフェースを提供し、開発者が異なるプロバイダー間で簡単に切り替えたり、コンポーネントを組み合わせたりできるようにします。また、アプリケーションの複雑さが増すにつれて、効率的に要素を接続するためのオーケストレーション機能や、アプリケーションの可観測性と評価をサポートする機能も備えています。\n\n全体として、LangChainは、開発者がAIアプリケーションを構築する際の利便性を高めるためのエコシステムを提供しています。'

### 6-4. 検索後の工夫

In [25]:
from langchain_core.documents import Document


def reciprocal_rank_fusion(
    retriever_outputs: list[list[Document]],
    k: int = 60,
) -> list[str]:
    # 各ドキュメントのコンテンツ (文字列) とそのスコアの対応を保持する辞書を準備
    content_score_mapping = {}

    # 検索クエリごとにループ（RFFアルゴリズムを使用）
    for docs in retriever_outputs:
        print(f"docs:{docs}")
        # 検索結果のドキュメントごとにループ
        for rank, doc in enumerate(docs):
            content = doc.page_content
            print(f"rank:{rank}")
            # 初めて登場したコンテンツの場合はスコアを0で初期化
            if content not in content_score_mapping:
                content_score_mapping[content] = 0

            # (1 / (順位 + k)) のスコアを加算
            content_score_mapping[content] += 1 / (rank + k)
            #print(f"content_score_mapping:{content_score_mapping}")

    # スコアの大きい順にソート
    ranked = sorted(content_score_mapping.items(), key=lambda x: x[1], reverse=True)  # noqa
    return [content for content, _ in ranked]

In [26]:
rag_fusion_chain = {
    "question": RunnablePassthrough(),
    "context": query_generation_chain | retriever.map() | reciprocal_rank_fusion,
} | prompt | model | StrOutputParser()

rag_fusion_chain.invoke("LangChainの概要を教えて")

docs:[Document(metadata={'file_name': 'why_langchain.mdx', 'file_path': 'docs\\docs\\concepts\\why_langchain.mdx', 'file_type': '.mdx', 'source': 'docs\\docs\\concepts\\why_langchain.mdx'}, page_content='# Why LangChain?\n\nThe goal of `langchain` the Python package and LangChain the company is to make it as easy as possible for developers to build applications that reason.\nWhile LangChain originally started as a single open source package, it has evolved into a company and a whole ecosystem.\nThis page will talk about the LangChain ecosystem as a whole.\nMost of the components within the LangChain ecosystem can be used by themselves - so if you feel particularly drawn to certain components but not others, that is totally fine! Pick and choose whichever components you like best for your own use case!\n\n## Features\n\nThere are several primary needs that LangChain aims to address:\n\n1. **Standardized component interfaces:** The growing number of [models](/docs/integrations/chat/) and

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。LangChainの目的は、開発者が推論を行うアプリケーションをできるだけ簡単に構築できるようにすることです。もともとはオープンソースのパッケージとして始まりましたが、現在は企業とエコシステム全体に進化しています。\n\nLangChainの主な特徴には以下が含まれます：\n\n1. **標準化されたコンポーネントインターフェース**: 様々なAIアプリケーション向けのモデルや関連コンポーネントの多様性に対処するため、LangChainは主要コンポーネントの標準インターフェースを提供し、プロバイダー間の切り替えを容易にします。\n\n2. **オーケストレーション**: 複数のコンポーネントやモデルを組み合わせて複雑なアプリケーションを構築するための効率的な接続を可能にします。LangGraphというライブラリを使用して、アプリケーションのフローをノードとエッジのセットとして表現できます。\n\n3. **可観測性と評価**: アプリケーションが複雑になるにつれて、内部で何が起こっているのかを理解することが難しくなります。LangSmithというプラットフォームを使用して、アプリケーションの監視や評価を行い、開発者が自信を持って迅速に質問に答えられるようにします。\n\nLangChainは、開発、運用、デプロイの各段階を簡素化し、開発者が自分のユースケースに最適なコンポーネントを選択して使用できるように設計されています。'

### 6-5. 複数の Retriever を使う工夫

In [ ]:
from langchain_community.retrievers import TavilySearchAPIRetriever

langchain_document_retriever = retriever.with_config(
    {"run_name": "langchain_document_retriever"}
)

web_retriever = TavilySearchAPIRetriever(k=3).with_config(
    {"run_name": "web_retriever"}
)

In [ ]:
from enum import Enum


class Route(str, Enum):
    langchain_document = "langchain_document"
    web = "web"


class RouteOutput(BaseModel):
    route: Route


route_prompt = ChatPromptTemplate.from_template("""\
質問に回答するために適切なRetrieverを選択してください。

質問: {question}
""")

route_chain = (
    route_prompt
    | model.with_structured_output(RouteOutput)
    | (lambda x: x.route)
)

In [ ]:
def routed_retriever(inp: dict[str, Any]) -> list[Document]:
    question = inp["question"]
    route = inp["route"]

    if route == Route.langchain_document:
        return langchain_document_retriever.invoke(question)
    elif route == Route.web:
        return web_retriever.invoke(question)

    raise ValueError(f"Unknown route: {route}")


route_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "route": route_chain,
    }
    | RunnablePassthrough.assign(context=routed_retriever)
    | prompt | model | StrOutputParser()
)

In [ ]:
route_rag_chain.invoke("LangChainの概要を教えて")

In [ ]:
route_rag_chain.invoke("東京の今日の天気は？")